# Resourcing: Solar panel cells

## Introduction ##
In this notebook we will use: 
- The Python open source Linear Programming library [PuLP](http://pythonhosted.org/PuLP/) to model and solve linear problems
- The Python open source data analysis library [Pandas] (https://pandas.pydata.org/) to show the results
- The IPython model display [Display] (https://ipython.org/ipython-doc/3/api/generated/IPython.display.html) to format the output 
- The Python numpy library [Numpy] (http://www.numpy.org/) to perform basic operations. 

We wil learn how to model problems in a way that scales to hundreds or thousands of variables and how to get all valuable information from the results generated by PuLP.

## Problem Definition

Weyland Corp is a British firm that manufactures solar panel cells. This firm supplies three differnt types of cell called: fast, normal and ultra. There are three operations involved in the manufacturing process. The following table describes the hours/month required to manufacture each model: 

**Table 1:** hours/month requirements

| Operation|   Fast (hours/cell) |   Normal (hours/cell) |  Ultra (hours/cell)| 
|----------|----------------------|------------------------|---------------------|
|  1       |        1             |           3            |        2            |
|  2       |        2             |           0            |        3            |
|  3       |        1             |           4            |        0            |

And the following table describes the total hours available for each operation:

**Table 2:** Total hours/month available

| Operation | Hours/Month |
| ----------| ------------|
|     1     |     400     |
|     2     |     600     |
|     3     |     600     |

The profit per each model is specified in the following table:
**Table 3:** Unit of profit per model

| Model     | Profit/Unit |
| ----------| ------------|
|  Fast   |     30      |
|  Normal |     20      |
|  Ultra   |     40      |

## Problem Model
### Decision variables
The decision variables are: 

$x_1:$Units of Fast cells / month

$x_2:$Units of Normal cells / month

$x_3:$Units of Ultra cells / month

### Objective Function
The objective function is:

$\max z=30x_1+20x_2+40x_3$

### Constraints
Subject to the following constraints:

$x_1+3x_2+2x_3 \leq  400$

$2x_1+0x_2+3x_3 \leq  600$

$x_1+4x_2+0x_3 \leq  600$

## Problem Solution with Pulp
Let us solve the problem with pulp. We start by importing PULP and instantiating the problem:

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import pulp

In [2]:
# Instantiate our problem class
model = pulp.LpProblem("Maximising profits for Weyland Corp", pulp.LpMaximize)

In this problem, we have three decision variables. We could name them individually like in the first example, but this does not scale (imagine we had hundreds). Instead we will use lists and tuple lists which can be really useful to minimise the problem configuration. 

First we create the lists:

In [3]:
# Construct our decision variable lists
cell_types = ['fast', 'normal', 'ultra']

The decision variables have the same characteristics (lower bound of 0, continuous variables). We can use the  LpVariable object's dict functionality to create the variables from the tuples. The good thing about tuples is that we could add more dimensions (eg cell categories) and still create all decision variables in one line of code.

In [4]:
cell_units = pulp.LpVariable.dicts("cells",
                                     (i for i in cell_types),
                                     lowBound=0,
                                     cat='Continuous')

PuLP provides an lpSum vector calculation for the sum of a list of linear expressions.  Although we only have 6 decision variables, we will use lpSum to construct the expression using this feature, again for the sake of scalibility.


In [5]:
# Technological Coefficients:
unit_profits = [30, 20, 40]
# Objective Function

model += (
    pulp.lpSum([
        unit_profits[i] * cell_units[cell_types[i]]
        for i in range(len(cell_types))])
)

Now we add the constraints using the tuplets to identify the coefficients

In [6]:
# Operations 
operations = ["Operation 1", "Operation 2", "Operation 3"]
# Constraints
A=[[1, 3, 2], #Coefficients of the first constraint
   [2, 0, 3], #Coefficients of the second constraint
   [1, 4, 0]] #Coefficients of the third constraint

# Capacities
b = [400, 600, 600]
 #Iterate matrix raws
for i in range(len(A)):
    model += pulp.lpSum([
        A[i][j] * cell_units[cell_types[j]] 
        for j in range(len(cell_types))]) <= b[i] , operations[i]    
#note that in this case all constraints are of type less or equal

In [7]:
# Solve our problem
model.solve(solver=pulp.solvers.GUROBI(msg = 0))

pulp.LpStatus[model.status]

Academic license - for non-commercial use only


'Optimal'

In [8]:
total_profit = pulp.value(model.objective)
display(Markdown("Total profit is %0.2f €"%total_profit))

display(Markdown("The following table shows the decision variables: "))
var_df = pd.DataFrame.from_dict(cell_units, orient="index", 
                                columns = ["Variables"])
var_df["Solution (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.X))
var_df["Reduced cost (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.RC))
var_df["Objective Coefficient (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.Obj))
var_df["Objective Lower bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjLow) if item.solverVar.SAObjLow > -0.1 else "-Inf" )
var_df["Objective Upper bound (GRB)"] = var_df["Variables"].apply(lambda item: "{:.2f}".format(item.solverVar.SAObjUp) if item.solverVar.SAObjUp != item.solverVar.UB else "Inf")


display(var_df)


const_dict = dict(model.constraints)
con_df = pd.DataFrame.from_records(list(const_dict.items()), exclude=["Expression"], columns=["Constraint", "Expression"])
con_df["Right Hand Side"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.RHS))
con_df["Shadow Price"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Pi))
con_df["Slack"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.Slack))
con_df["Min RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSLow) )
con_df["Max RHS"]=con_df["Constraint"].apply(lambda item: "{:.2f}".format(const_dict[item].solverConstraint.SARHSUp) if const_dict[item].solverConstraint.SARHSUp < 1e10 else "Inf" )


display(Markdown("The following table shows the constraints: "))
display(con_df)

Total profit is 9666.67 €

The following table shows the decision variables: 

,Variables,Solution (GRB),Reduced cost (GRB),Objective Coefficient (GRB),Objective Lower bound (GRB),Objective Upper bound (GRB)
fast,cells_fast,300.00,0.00,30.00,24.44,Inf
normal,cells_normal,33.33,0.00,20.00,-0.00,90.00
ultra,cells_ultra,0.00,-8.33,40.00,-Inf,48.33


The following table shows the constraints: 

,Constraint,Right Hand Side,Shadow Price,Slack,Min RHS,Max RHS
0,Operation_1,400.00,6.67,0.00,300.00,525.00
1,Operation_2,600.00,11.67,0.00,0.00,800.00
2,Operation_3,600.00,0.00,166.67,433.33,Inf


### Question 1
Peter Weyland believes that, for the reputation of the company, Weyland should manufacture at least 40 ultra cells per month. If this plan goes ahead, how would it affect the current profit?

In [9]:
display(Markdown("**Answer**"))
answer_1 = -40*cell_units["ultra"].solverVar.RC;
display(Markdown("40 extra units of ultra would lower the profits: **%0.3f€**"%answer_1))

**Answer**

40 extra units of ultra would lower the profits: **333.333€**

### Question 2
The engineer informed management that due to maintenance operations, the capacities of Operation 1, 2, and 3 have changed to 360, 600 and 500 respectively. How would this affect the profit?

In [10]:
display(Markdown("**Answer**"))
display(Markdown("We need to check the shadow price of Operation 1 and 2. Operation 3 has a surplus of capacity and the minimum RHS is below the new capacity, so it will have no impact."))

b2 = [360, 600, 500]
maintenance = np.subtract(b,b2)
answer_2=np.dot(maintenance, pd.to_numeric(con_df["Shadow Price"]))


display(Markdown("The profit would be reduced in **%0.2f** € per month"%answer_2))


**Answer**

We need to check the shadow price of Operation 1 and 2. Operation 3 has a surplus of capacity and the minimum RHS is below the new capacity, so it will have no impact.

The profit would be reduced in **266.80** € per month

### Question 3
Mr. Weyland insisted that the company needs to change the strategy towards producing ultra fast cells to position Weyland Corp in this segment. The company reviewed the prices of the ultra fast cells and changed them to 30€, 10€ and 50€ per fast, normal and ultra cells. With the new prices, would it be profitable to produce ultra fast cells?

In [11]:
display(Markdown("**Answer**"))
display(Markdown("We need to verify the bounds for the objective coefficients for the three variables.") )
display(Markdown("First we check if the new coefficients of the objective function would change the basic solution."))
display(Markdown("the variables that change the basic solution are:"))                 
new_profits = [30, 10, 50]
entering_df = var_df[(new_profits < pd.to_numeric(var_df["Objective Lower bound (GRB)"])) | (new_profits > pd.to_numeric(var_df["Objective Upper bound (GRB)"]))]
display(entering_df)

answer_3 = "It is profitable to produce ultra fast cells with the new prices" if ("ultra" in entering_df.index) else "It is not profitable to produce ultra fast cells with the new prices"
display(Markdown(answer_3))

**Answer**

We need to verify the bounds for the objective coefficients for the three variables.

First we check if the new coefficients of the objective function would change the basic solution.

the variables that change the basic solution are:

,Variables,Solution (GRB),Reduced cost (GRB),Objective Coefficient (GRB),Objective Lower bound (GRB),Objective Upper bound (GRB)
ultra,cells_ultra,0.00,-8.33,40.00,-Inf,48.33


It is profitable to produce ultra fast cells with the new prices

### Question 4
What is the impact in the new profit if we opted for this new prices?

In [12]:
display(Markdown("**Answer**"))
display(Markdown("We need to define a new model (model 2) for the reviewed price."))

model2 = pulp.LpProblem("Updated model for Weyland Corp", pulp.LpMaximize)
# Technological Coefficients:
unit_profits2 = [30, 10, 50]

# Objective Function
model2 += (
    pulp.lpSum([
        unit_profits2[i] * cell_units[cell_types[i]]
        for i in range(len(cell_types))])
)

#Iterate matrix raws
#Since there is no change in the capacities, we can use the same constraints
for i in range(len(A)):
    model2 += pulp.lpSum([
        A[i][j] * cell_units[cell_types[j]] 
        for j in range(len(cell_types))]) <= b[i] , operations[i]    

#note that in this case all constraints are of type less or equal
model2.solve()
pulp.LpStatus[model2.status]
total_profit2 = pulp.value(model2.objective)

display(Markdown("The total profit is **%.2f** € per month"%total_profit2))

answer_4 = total_profit2 - total_profit 
display(Markdown("The profit would change in **%0.2f** € per month"%answer_4))


**Answer**

We need to define a new model (model 2) for the reviewed price.

The total profit is **10000.00** € per month

The profit would change in **333.33** € per month

#### Question 5
The engineer is studying a new type of fast cells that would require 2 hours of operation 1, 0 hours of operation 2 and 1 hour of operation 3. The estimated market price is 20€. How would this affect the production plan?

In [13]:
display(Markdown("**Answer**"))
display(Markdown("We need to define a new model (model 3) for the new standard cell type."))

model3 = pulp.LpProblem("Updated model for Weyland Corp II", pulp.LpMaximize)
cell_types3 = ['fast', 'standard', 'normal', 'ultra']

# Technological Coefficients:
unit_profits3 = [30, 20, 20, 40]

# New variables
cell_units3 = pulp.LpVariable.dicts("cells",
                                     (i for i in cell_types3),
                                     lowBound=0,
                                     cat='Continuous')
# Objective Function
model3 += (
    pulp.lpSum([
        unit_profits3[i] * cell_units3[cell_types3[i]]
        for i in range(len(cell_types3))])
)
#Update the constraints
# Constraints
A=[[1, 2, 3, 2], #Coefficients of the first constraint
   [2, 0, 0, 3], #Coefficients of the second constraint
   [1, 2, 4, 0]] #Coefficients of the third constraint

# Capacities
b = [400, 600, 600]

#Iterate matrix raws
for i in range(len(A)):
    model3 += pulp.lpSum([
        A[i][j] * cell_units3[cell_types3[j]] 
        for j in range(len(cell_types3))]) <= b[i] , operations[i]    

model3.solve()
pulp.LpStatus[model3.status]
    
#note that in this case all constraints are of type less or equal
total_profit3 = pulp.value(model3.objective)

display(Markdown("The total profit is **%.2f** € per month"%total_profit3))

# Print our decision variable values
display(Markdown("The following tables show the values obtained: "))
variable_names3 = []
values3 = []
opportunity_costs3 = []
for v in model3.variables():
    variable_names3.append(v.name)
    values3.append(v.varValue)
    opportunity_costs3.append(v.dj)
    
variable_raw_data3 = {"Decision Variable": variable_names3,
                     "Solution value": values3,
                     "Unit profit": unit_profits3,
                     "Total Contribution":np.multiply(unit_profits3,values3),
                    "Oportunity cost": opportunity_costs3}

var_df3 = pd.DataFrame(variable_raw_data3)
display(var_df3)

# Print out the values of the constraints
constraint_names3 = []
constants3 = []
senses3 = []
pis3 = []
slacks3 = []
modifieds3 = []
for name, c in list(model3.constraints.items()):
    constraint_names3.append(name)
    constants3.append(-c.constant)
    senses3.append(c.sense)
    pis3.append(c.pi)
    slacks3.append(c.slack)
    modifieds3.append(c.modified)

display(Markdown("The following table shows the problem constraints: "))

constraint_raw_data3 = {"Constraint":constraint_names3,
            "Left-hand side":np.subtract(constants3,slacks3),
            "Direction":senses3,
            "Right-hand side":constants3,
            "Slack": slacks3,
            "Shadow price": pis3
            }

const_df3 = pd.DataFrame(constraint_raw_data3)
display(const_df3)

answer_4 = total_profit2 - total_profit 
display(Markdown("The profit would change in **%0.2f** € per month"%answer_4))



**Answer**

We need to define a new model (model 3) for the new standard cell type.

The total profit is **10000.00** € per month

The following tables show the values obtained: 

,Decision Variable,Solution value,Unit profit,Total Contribution,Oportunity cost
0,cells_fast,300.0,30,9000.0,-0.0
1,cells_normal,0.0,20,0.0,-10.0
2,cells_standard,50.0,20,1000.0,-0.0
3,cells_ultra,0.0,40,0.0,-10.0


The following table shows the problem constraints: 

,Constraint,Left-hand side,Direction,Right-hand side,Slack,Shadow price
0,Operation_1,400.0,-1,400,-0.0,10.0
1,Operation_2,600.0,-1,600,-0.0,10.0
2,Operation_3,400.0,-1,600,200.0,-0.0


The profit would change in **333.33** € per month

## Question 6
Mr. Weyland thinks that the operation time for Operation 2 for the normal cells can be cut to only 3h. How would this improve the production plan?

**Answer**

Reducing the operational time for operation 2 3 hours would increase/reduce the profit: **-0.000€**